In [ ]:
def random_warp(img):
 
    #getting rows, cols form the input img
    r,c,_ = img.shape
    #degree of warping
    degree = 0.09
    # random scaling coefficients
    rndx = np.random.rand(3) - 0.5
    rndx *= c * degree
    rndy = np.random.rand(3) - 0.5
    rndy *= r * degree
    # 3 starting points for transform, 1/4 way from edges
    x1 = c/4
    x2 = 3*c/4
    y1 = r/4
    y2 = 3*r/4

    pts1 = np.float32([[y1,x1],[y2,x1],[y1,x2]])
    pts2 = np.float32([[y1+rndy[0],x1+rndx[0]],[y2+rndy[1],x1+rndx[1]],[y1+rndy[2],x2+rndx[2]]])
    
    M = cv2.getAffineTransform(pts1,pts2)
    dst = cv2.warpAffine(img,M,(c,r))   
    dst = dst[:,:,np.newaxis]   
    return dst

Testing random_warp:

In [ ]:
test_img = X_train_norm[random.randint(0,len(X_train_norm))]
test_dst = random_warp(test_img)
fig, axes = plt.subplots(1,2, figsize=(7, 3))

axes[0].axis('off')
axes[0].imshow(test_img.squeeze(), cmap='gray')
axes[0].set_title('original')

axes[1].axis('off')
axes[1].imshow(test_dst.squeeze(), cmap='gray')
axes[1].set_title('warped')

print('shape input/output:', test_img.shape,"/", test_dst.shape)

#### Third function: random_zoom

In [ ]:
def random_zoom(img): 
    #getting rows, cols form the input img
    r,c,_ = img.shape
    # zooming factor
    px = np.random.randint(-4,4)
    # ending locations
    pts1 = np.float32([[px,px],[r-px,px],[px,c-px],[r-px,c-px]])
    # starting locations (4 corners)
    pts2 = np.float32([[0,0],[r,0],[0,c],[r,c]])
    M = cv2.getPerspectiveTransform(pts1,pts2)
    dst = cv2.warpPerspective(img,M,(r,c))  
    dst = dst[:,:,np.newaxis]   
    return dst

Testing random_zoom:

In [ ]:
test_img = X_train_norm[random.randint(0,len(X_train_norm))]
test_dst = random_zoom(test_img)
fig, axes = plt.subplots(1,2, figsize=(7, 3))

axes[0].axis('off')
axes[0].imshow(test_img.squeeze(), cmap='gray')
axes[0].set_title('original')

axes[1].axis('off')
axes[1].imshow(test_dst.squeeze(), cmap='gray')
axes[1].set_title('zoomed')

print('shape input/output:', test_img.shape,"/", test_dst.shape)

### Stacking up the dataset

In [ ]:
input_indices = []
output_indices = []

for i in range(n_classes):
    class_indices = np.where(y_train == i)
    n_samples = len(class_indices[0])
    if n_samples < 1000:
        for j in range(1000 - n_samples):
            input_indices.append(class_indices[0][j%n_samples])
            output_indices.append(X_train_norm.shape[0])
            new_img = X_train_norm[class_indices[0][j % n_samples]]
            new_img = random_translate(random_zoom(random_warp(new_img)))
            #combining 
            X_train_norm = np.concatenate((X_train_norm, [new_img]), axis=0)
            y_train = np.concatenate((y_train, [i]), axis=0)

print('X/y shapes after augmentation:', X_train_norm.shape, y_train.shape)   

In [ ]:
# show comparisons of 5 random augmented data points
choices = list(range(len(input_indices)))
picks = []
for i in range(5):
    rnd_index = np.random.randint(low=0,high=len(choices))
    picks.append(choices.pop(rnd_index))
fig, axes = plt.subplots(2,5, figsize=(15, 6))
fig.subplots_adjust(hspace = .2, wspace=.001)
axes = axes.ravel()
for i in range(5):
    image = X_train_norm[input_indices[picks[i]]].squeeze()
    axes[i].axis('off')
    axes[i].imshow(image, cmap = 'gray')
    axes[i].set_title(y_train[input_indices[picks[i]]])
for i in range(5):
    image = X_train_norm[output_indices[picks[i]]].squeeze()
    axes[i+5].axis('off')
    axes[i+5].imshow(image, cmap = 'gray')
    axes[i+5].set_title(y_train[output_indices[picks[i]]])
    print(image.shape)


In [ ]:
his = np.histogram(y_train,bins=n_classes)
fig, axes = plt.subplots(1,1, figsize= (12,4))
plt.bar(his[1][1:],his[0], width = 0.8)
axes.set_xticks(his[1][1:])
axes.set_xticklabels(i for i in range(43))
plt.axhline(y=1000, color='r', linestyle='--')
plt.show()

Splitting into train und validation set with [sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

### Model Architecture

In [ ]:
### Define your architecture here.
### Feel free to use as many code cells as needed.

In [ ]:
import tensorflow as tf

EPOCHS = 100
BATCH_SIZE = 190

In [ ]:
from tensorflow.contrib.layers import flatten

def LeNet(x):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    #keep_prob = 0.75
    # Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x20. 
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 1, 20), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(20))
    conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b
    # Activation.
    conv1 = tf.nn.relu(conv1)
    # Pooling. Input = 28x28x20. Output = 14x14x20.
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    # Layer 2: Convolutional. Output = 10x10x40.  
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 20, 40), mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(40))
    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b    
    # Activation.
    conv2 = tf.nn.relu(conv2)
    # Pooling. Input = 10x10x40. Output = 5x5x40.
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    # Flatten. Input = 5x5x40. Output = 1000.
    fc0   = flatten(conv2)    
    # Layer 3: Fully Connected. Input = 1000. Output = 325.
    fc1_W = tf.Variable(tf.truncated_normal(shape=(1000, 325), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(325))
    fc1   = tf.matmul(fc0, fc1_W) + fc1_b   
    # Activation.
    fc1    = tf.nn.relu(fc1)
    #Dropout
    fc1 = tf.nn.dropout(fc1, keep_prob)
    # Layer 4: Fully Connected. Input = 325. Output = 175.
    fc2_W  = tf.Variable(tf.truncated_normal(shape=(325, 175), mean = mu, stddev = sigma))
    fc2_b  = tf.Variable(tf.zeros(175))
    fc2    = tf.matmul(fc1, fc2_W) + fc2_b
    # Activation.
    fc2    = tf.nn.relu(fc2)
    # Dropout
    fc2 = tf.nn.dropout(fc2, keep_prob)
    # Layer 5: Fully Connected. Input = 175. Output = 43.
    fc3_W  = tf.Variable(tf.truncated_normal(shape=(175, 43), mean = mu, stddev = sigma))
    fc3_b  = tf.Variable(tf.zeros(43))
    logits = tf.matmul(fc2, fc3_W) + fc3_b    
    return logits

In [ ]:
x = tf.placeholder(tf.float32, (None, 32, 32, 1))
y = tf.placeholder(tf.int32, (None))
keep_prob = tf.placeholder(tf.float32)
one_hot_y = tf.one_hot(y, 43)

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.

In [ ]:
rate = 0.0009

logits = LeNet(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

In [ ]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.0})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    Epochs_plot = []
    Acc = []
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 0.5})
            
        validation_accuracy = evaluate(X_valid, y_valid)
        training_accuracy = evaluate(X_train, y_train)
        if i%10 == 0:
            print("EPOCH {} ...".format(i+1))
            print("Training Accuracy = {:.3f}".format(training_accuracy))
            print("Validation Accuracy = {:.3f}".format(validation_accuracy))
            print()
        Epochs_plot.append(i+1)
        Acc.append(validation_accuracy)
    if validation_accuracy > 0.990:
        saver.save(sess, './lenet_v06')
        print("Model saved")
#save Acc for Documentation
fig = plt.figure(figsize=(5,3))
plt.plot(Acc)
fig.savefig('Documentation/optimum1.png', dpi=fig.dpi)

### Testing on the Test Set

In [ ]:
import numpy as np
import pandas as pd

n_train = len(X_train)
n_validation = len(X_valid)
n_test = len(X_test)
image_shape = X_train[0].shape
n_classes = len(np.unique(y_train))

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)
print(len(X_valid))

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. 

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

In [ ]:
import matplotlib.pyplot as plt
import random
%matplotlib inline

fig, axes = plt.subplots(3,5, figsize=(15, 6))
fig.subplots_adjust(hspace = .2, wspace=.001)
axes = axes.ravel()
for i in range(15):
    index = random.randint(0, len(X_train))
    img = X_train[index]
    axes[i].axis('off')
    axes[i].imshow(img)
    axes[i].set_title(y_train[index])

In [ ]:
import pandas as pd 
names = pd.read_csv("signnames.csv")
names.head()

distribiution is not optimal. trying to stock up to at least 1000 images each class.

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. 

Other pre-processing steps are optional. You can try different techniques to see if it improves performance. 

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:
X_train_rgb = np.copy(X_train)
X_test_rgb = np.copy(X_test)
X_valid_rgb = np.copy(X_valid)

Converting to grayscale

In [ ]:
def grayscale(x):
    return np.sum(x/3, axis=3, keepdims=True)

In [ ]:
X_train_gry = grayscale(X_train)
X_test_gry = grayscale(X_test)
X_valid_gry = grayscale(X_valid)

In [ ]:
# Visualize rgb vs grayscale

offset = 2017
fig, axes = plt.subplots(8,10, figsize=(15, 11))
fig.subplots_adjust(hspace = .1, wspace=.001)
axes = axes.ravel()
for j in range(0,8,2):
    for i in range(10):
        index = i + j*10
        image = X_train_rgb[index + offset]
        axes[index].axis('off')
        axes[index].imshow(image)
    for i in range(10):
        index = i + j*10 + 10 
        image = X_train_gry[index + offset - 10].squeeze()
        axes[index].axis('off')
        axes[index].imshow(image, cmap='gray')

In [ ]:
#Normalize the Images
def normalize(x):
    return (x - 128)/128 

In [ ]:
X_train_norm = normalize(X_train_gry)
X_test_norm = normalize(X_test_gry)
X_valid_norm = normalize(X_valid_gry)

In [ ]:
# Compare normalized and gray image
fig, axes = plt.subplots(1,2, figsize=(7, 3))
axes = axes.ravel()


axes[0].axis('off')
axes[0].set_title('original_gray')
axes[0].imshow(X_train_gry[0].squeeze(), cmap='gray')

axes[1].axis('off')
axes[1].set_title('normalized_img')
axes[1].imshow(X_train_norm[0].squeeze(), cmap='gray')

print("Shape gray Train-img: ", X_train_gry.shape,", ", "Mean: ", np.mean(X_train_gry))
print("Shape normalized Train-img: ", X_train_norm.shape,", ", "Mean: ", np.mean(X_train_norm))

### Augmentation
Augmenting the Training Dataset to at least 1000 examples each class.
The augmentation will be applied by 3 functions.
1. random_translate 
2. random_warp
3. random_scale

[Useful cv2 functions](https://docs.opencv.org/trunk/da/d6e/tutorial_py_geometric_transformations.html)

#### First function for augmenting: random_translate